In [ ]:
!wget https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip

--2021-03-11 11:37:59--  https://raw.githubusercontent.com/sagihaider/CE888_2021/main/Lab_8/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20194537 (19M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  19.26M  76.5MB/s    in 0.3s    

2021-03-11 11:38:00 (76.5 MB/s) - ‘data.zip’ saved [20194537/20194537]



In [ ]:
!unzip data.zip

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
path_humans = './data/Humans'
path_cats = './data/cats'
path_dogs = './data/dogs/'
path_horses = './data/horses/'

humans = os.listdir(path_humans)
cats = os.listdir(path_cats)
dogs = os.listdir(path_dogs)
horses = os.listdir(path_horses)

human_img_sizes = []
cats_img_sizes = []
dogs_img_sizes = []
horses_img_sizes = []

for i in range(202):
  hu = plt.imread(os.path.join(path_humans, humans[i]))
  c = plt.imread(os.path.join(path_cats, cats[i]))
  d = plt.imread(os.path.join(path_dogs, dogs[i]))
  ho = plt.imread(os.path.join(path_horses, horses[i]))
  human_img_sizes.append(hu.shape[:-1])
  cats_img_sizes.append(c.shape[:-1])
  dogs_img_sizes.append(d.shape[:-1])
  horses_img_sizes.append(ho.shape[:-1])


In [ ]:

mean_sizes = [np.mean(human_img_sizes, axis=0), np.mean(cats_img_sizes, axis=0),
              np.mean(dogs_img_sizes, axis=0), np.mean(horses_img_sizes, axis=0)]

target_size = np.array(mean_sizes).mean(axis=0)
target_size

array([304.6200495 , 329.92574257])

In [ ]:
image_gen = ImageDataGenerator(rescale=1.0/255, 
                               rotation_range=20,
                               horizontal_flip=True, 
                               validation_split=0.1)

In [ ]:
train = image_gen.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='training')

val = image_gen.flow_from_directory('./data/', target_size=(304,329),
                                      batch_size=4,subset='validation')

Found 728 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras import Model

img_size = [304,329,3]

In [ ]:
input = Input(shape = img_size)
model = ResNet50(input_tensor=input,)

102973440/102967424 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 310, 335, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 152, 165, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 152, 165, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [ ]:
last_layer = model.get_layer('avg_pool').output
out = Dense(4, activation='softmax', name='output')(last_layer)
model = Model(input, out)

for layer in model.layers[:-1]:
  layer.trainable = False


model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 304, 329, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 310, 335, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 152, 165, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 152, 165, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics="accuracy")

In [ ]:
my_callbacks = [
    EarlyStopping(monitor="val_accuracy", patience=10, restore_best_weights=True),
    ModelCheckpoint(filepath='vgg16_model.h5', save_best_only=True),
]

In [ ]:
model.fit(train, validation_data=val, epochs=50, callbacks=my_callbacks)

Epoch 1/50
182/182 [==============================] - 57s 122ms/step - loss: 1.5644 - accuracy: 0.3117 - val_loss: 1.4548 - val_accuracy: 0.2500
Epoch 2/50
182/182 [==============================] - 21s 116ms/step - loss: 1.3832 - accuracy: 0.3595 - val_loss: 1.4275 - val_accuracy: 0.3375
Epoch 3/50
182/182 [==============================] - 21s 115ms/step - loss: 1.3570 - accuracy: 0.3680 - val_loss: 1.3388 - val_accuracy: 0.3500
Epoch 4/50
182/182 [==============================] - 21s 114ms/step - loss: 1.3167 - accuracy: 0.3555 - val_loss: 1.1942 - val_accuracy: 0.4625
Epoch 5/50
182/182 [==============================] - 21s 115ms/step - loss: 1.2476 - accuracy: 0.4417 - val_loss: 1.2942 - val_accuracy: 0.2750
Epoch 6/50
182/182 [==============================] - 21s 113ms/step - loss: 1.2439 - accuracy: 0.4060 - val_loss: 1.1843 - val_accuracy: 0.4000
Epoch 7/50
182/182 [==============================] - 21s 115ms/step - loss: 1.2198 - accuracy: 0.4052 - val_loss: 1.1319 - val_ac

In [ ]:
model.history.history

{'accuracy': [0.31593406200408936,
  0.35164836049079895,
  0.3859890103340149,
  0.3887362778186798,
  0.4079670310020447,
  0.40247252583503723,
  0.4065934121608734,
  0.42032966017723083,
  0.4134615361690521,
  0.43406593799591064,
  0.450549453496933,
  0.4313186705112457,
  0.450549453496933,
  0.4313186705112457,
  0.47664836049079895,
  0.4601648449897766,
  0.47252747416496277,
  0.4464285671710968,
  0.45604395866394043,
  0.4175824224948883,
  0.46978020668029785,
  0.4642857015132904,
  0.4684065878391266,
  0.4381868243217468,
  0.4313186705112457,
  0.5054945349693298,
  0.49862638115882874,
  0.49862638115882874,
  0.47252747416496277],
 'loss': [1.4503775835037231,
  1.355900764465332,
  1.325276494026184,
  1.2763556241989136,
  1.2797671556472778,
  1.274245262145996,
  1.2336369752883911,
  1.2307902574539185,
  1.258833408355713,
  1.2455735206604004,
  1.1868846416473389,
  1.2035595178604126,
  1.1992456912994385,
  1.2054884433746338,
  1.1915425062179565,
  1.1